In [20]:
import os
import json
import logging
from IPython.display import clear_output
from collections import OrderedDict as odict

from marshaltools import ProgramList

from utils import get_config

In [5]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2018-10-01'

logpath = 'logs/'

# log
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_sedm_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [54]:
# allocation = {
#     ztfname1: {
#         telescope1: [date, status],
#         telescope2: [...]}
#     ztfname2: {...}
# }

targets = json.load(open('sne_following.json'))
allocation = json.load(open('sne_allocation.json'))
pl = ProgramList('AMPEL Test')

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 10)
INFO:marshaltools.ProgramList:Loaded 214 saved sources for program AMPEL Test.


In [16]:
# Step 1: check whether any previously allocated target is now typed on the marshal. If so mark it done.
logger.info('Step 1: Automatic type query')
for snname, info in allocation.items():
    if 'sedm' in info.keys():
        cl = pl.sources[snname]['classification']
        if cl is not None:
            info['sedm'][1] = 'done'
            logger.info('Source %s has classification %s on the marshal now. Marking it as done.'%(snname, cl))

In [56]:
options = odict()
options['1'] = ['in_progress', 'in progress, i.e. in SEDM queue likely to be observed']
options['2'] = ['failed', 'failed, i.e. obs not good, too faint or too low priority'] 
options['3'] = ['done', 'done, i.e a spectrum was observed by some telescope showing it either to be a SNIa or not']
options['4'] = ['recheck', 'recheck, i.e. there is a spectrum but it is not typed (check later whether more spectra are warranted)']
options['0'] = ['skip', 'skip, e.g. because it was not scheduled for SEDM yet']

def input_prompt(snname, idx=None, options=options):
    if idx is None:
        print('Please check the SEDM status of target %s:'%snname)
    else:
        print('Please check the SEDM status of target %s (%s):'%(snname, idx))
    print('http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s\n'%snname)
    for k, v in options.items():
        print('(%s) %s'%(k, v[1]))
    i_ = input('Please select one of the options: ')
    if i_ in options.keys():
        return options[i_][0]
    else:
        return None

In [34]:
# Step 2: check targets previously marked as SEDM targets for their progress
# NOTE: Usage of SEDM do denote these targets' status is slightly misleading because it includes everything
#       reported on the marshal
logger.info('Step 2: Rechecking sources previously allocated to SEDM')
for k, (snname, info) in enumerate(allocation.items()):
    if 'sedm' in info.keys() and info['sedm'][1] != 'done':
        done = False
        while not done:
            o_ = input_prompt(snname)
            if o_ == 'skip':
                logger.info('Source %s skipped.'%(snname))
                done = True
            elif o_ is not None:
                info['sedm'] = [date, o_]
                logger.info('Source %s updated to %s.'%(snname, o_))
                done = True
            else:
                clear_output()
                print('Error: Please select one of the options.')

In [42]:
# Step 3: Go through targets not in the list and check their status
# Skip sources that are not scheduled for SEDM
logger.info('Step 3: Checking new sources')
for snname in targets.keys():
    if snname not in allocation.keys():
        cl = pl.sources[snname]['classification']
        if cl is not None:
            if snname not in allocation:
                allocation[snname] = {}
            allocation[snname]['sedm'] = [date, 'done']
            logger.info('Source %s has classification %s on the marshal now. Marking it as done.'%(snname, cl))
        else:
            done = False
            while not done:
                o_ = input_prompt(snname)
                if o_ == 'skip':
                    logger.info('Source %s skipped.'%(snname))
                    done = True
                    clear_output()
                elif o_ is not None:
                    if snname not in allocation:
                        allocation[snname] = {}
                    allocation[snname]['sedm'] = [date, o_]
                    logger.info('Source %s added (status: %s).'%(snname, o_))
                    done = True
                    clear_output()
                else:
                    clear_output()
                    print('Error: Please select one of the options.')

INFO:ufeindt:Step 3: Checking new sources


Please re-check the SEDM status of target ZTF18abcqvhs:
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abcqvhs

(1) in progress, i.e. in SEDM queue likely to be observed
(2) failed, i.e. obs not good, too faint or too low priority
(3) done, i.e a spectrum was observed by some telescope showing it either to be a SNIa or not
(0) skip
Please select one of the options above: 0


INFO:ufeindt:Source ZTF18abcqvhs skipped.


Please re-check the SEDM status of target ZTF18abnujuy:
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abnujuy

(1) in progress, i.e. in SEDM queue likely to be observed
(2) failed, i.e. obs not good, too faint or too low priority
(3) done, i.e a spectrum was observed by some telescope showing it either to be a SNIa or not
(0) skip
Please select one of the options above: 0


INFO:ufeindt:Source ZTF18abnujuy skipped.


...

INFO:ufeindt:Source ZTF18abxwiqz added (status: in_progress).


Please re-check the SEDM status of target ZTF18abxbhov:
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18abxbhov

(1) in progress, i.e. in SEDM queue likely to be observed
(2) failed, i.e. obs not good, too faint or too low priority
(3) done, i.e a spectrum was observed by some telescope showing it either to be a SNIa or not
(0) skip
Please select one of the options above: 2


INFO:ufeindt:Source ZTF18abxbhov added (status: failed).
INFO:ufeindt:Source ZTF18abvtrnc has classification SN Ia on the marshal now. Marking it as done.


Please re-check the SEDM status of target ZTF17aabihdn:
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF17aabihdn

(1) in progress, i.e. in SEDM queue likely to be observed
(2) failed, i.e. obs not good, too faint or too low priority
(3) done, i.e a spectrum was observed by some telescope showing it either to be a SNIa or not
(0) skip
Please select one of the options above: 0


INFO:ufeindt:Source ZTF17aabihdn skipped.


In [67]:
json.dump(allocation, open('sne_allocation.json', 'w'))